## Parallel Processing

Launching multiple copies if Venner command line using veneer-py


Running simulations in parallel

In [1]:
import veneer 
import numpy as np
import pandas as pd
import chaospy as cp
import matplotlib.pyplot as plt
# import SALib
import SALib
from SALib.sample import sobol_sequence
%matplotlib inline

ModuleNotFoundError: No module named 'chaospy'

In [ ]:
%load_ext autoreload
%autoreload 2
import source_runner as sr
from source_runner import *

In [3]:
#### Run this to improve model performance, mainly through parallel computing. These can also be modified through Source UI
def configure_options(self,options):
    lines = ["# Generated Script","from Dynamic_SedNet.PluginSetup import DSScenarioDetails"]
    lines += ["DSScenarioDetails.%s = %s"%(k,v) for (k,v) in options.items()]
    script = '\n'.join(lines)
    #print(script)
    res = self.model._safe_run(script)

In [4]:
from veneer.manage import create_command_line
from veneer.manage import start,kill_all_now

In [18]:
veneer_install = 'd:/UserData/wangd/Source_files/Source 4.5.0.9189/Plugins/CommunityPlugins/'
source_version = '4.5.0'
cmd_directory = '../veneer_cmd'
veneer_cmd = create_command_line(veneer_install,source_version,dest=cmd_directory)
veneer_cmd

WindowsPath('../veneer_cmd/FlowMatters.Source.VeneerCmd.exe')

In [19]:
project = 'D:/UserData/wangd/Source_files/MW Source proj files April2020/RC10_MW_models/models/MW_BASE_RC10.rsproj'
# generate copies of servers
num_copies=1
first_port=9877
processes, ports = start(project,n_instances=num_copies,ports=first_port,debug=True, remote=False, veneer_exe=veneer_cmd)
vs = [veneer.Veneer(port=p) for p in ports]

Starting ..\veneer_cmd\FlowMatters.Source.VeneerCmd.exe -p 9877 -s  "D:\UserData\wangd\Source_files\MW Source proj files April2020\RC10_MW_models\models\MW_BASE_RC10.rsproj"
[0] Loading plugins

[0] Loaded .\.\FlowMatters.Source.Veneer.dll

[0] Loaded .\.\CodeProjectWizardDemo.dll

[0] Loaded .\.\Dynamic_SedNet.dll

[0] Loaded .\.\GBR_DynSed_Extension.dll

[0] Plugins loaded (4/4)

[0] Opening project file: D:\UserData\wangd\Source_files\MW Source proj files April2020\RC10_MW_models\models\MW_BASE_RC10.rsproj

ERROR[0] log4net:ERROR Failed to find configuration section 'log4net' in the application's .config file. Check your .config file for the <log4net> and <configSections> elements. The configuration section should look like: <section name="log4net" type="log4net.Config.Log4NetConfigurationSectionHandler,log4net" />

[0] Loading project

[0] Project Loaded

[0] RC10

[0] RC10

[0] [8:34:24 PM] Veneer, by Flow Matters: http://www.flowmatters.com.au

[0] [8:34:24 PM] Started Source RES

Configure model for runs

#Start

In [20]:
# set model configuration
for v in vs:
    configure_options(v,{'RunNetworksInParallel':True,'PreRunCatchments':True,'ParallelFlowPhase':True})
    v.model.sourceScenarioOptions("PerformanceConfiguration","ProcessCatchmentsInParallel",True)
    ### Run this to turn off dsednet reporting window
    configure_options(v,{'ShowResultsAfterRun':False,'OverwriteResults':True})

In [21]:
# Set model runs
# Define start and end time, with a half year warmup period
start_date = '01/07/1998'
end_date = '30/06/2014'
record_gauge = 'gauge_124001B_AndromacheRvJochheims'
record_var = 'Constituents@Sediment - Fine@Downstream Flow Mass'
filter_elements = set_filter(veneer, record_gauge, record_var)

In [22]:
# Configuring recording and retrieving results
ts_match_criteria = {'RecordingVariable':'Constituents@Sediment - Fine@Downstream Flow Mass',
                    'NetworkElement':'gauge_124001B_OConnellRvStaffordsCrossing'}
for v in vs:
    v.configure_recording(enable=[ts_match_criteria])

#End

Parameter process 

#Start

In [10]:
# assign distributions to parameters
param_file = '../input/parameter-corrected.csv'
parameters = sr.load_parameter_file(param_file) 
# parameters.drop(axis=1, columns=parameters.columns[0], inplace=True)
param_names, param_vename_dic, param_vename, param_types = sr.group_parameters(parameters)
initial_params = get_initial_param_vals(vs[0], param_names, param_vename, param_vename_dic)

In [11]:
parameters

,Veneer_name,Veneer_location,min,max,upper,type,distribution,short_name,Unnamed: 8
0,BankFullFlow,v.model.link.constituents,0.700,1.300,9999999,1,uniform,BFF,NaN
1,bankErosionCoeff,v.model.link.constituents,0.500,1.500,9999999,1,uniform,BEC,NaN
2,BankHeight_M,v.model.link.constituents,0.850,1.150,9999999,1,uniform,BHM,NaN
3,bankMgtFactor,v.model.link.constituents,0.900,1.100,9999999,1,uniform,BMF,NaN
4,eventMeanConcentration,v.model.catchment.generation,0.800,1.200,9999999,1,uniform,EMC,NaN
5,HillSlopeFinePerc,v.model.catchment.generation,0.950,1.050,100,1,uniform,HSFP,NaN
6,HillslopeFineSDR,v.model.catchment.generation,0.500,1.500,100,1,uniform,HFSDR,NaN
7,Load_Conversion_Factor,v.model.catchment.generation,0.900,1.000,1,1,uniform,LCF,NaN
8,RiparianVegPercent,v.model.link.constituents,0.900,1.100,100,1,uniform,RVP,NaN
9,sedBulkDensity,v.model.link.constituents,0.950,1.050,9999999,1,uniform,SBD,NaN


Generate samples with Sobol' sampling

In [12]:
len_params = len(parameters)
num_runs = 552
sobol_se = sobol_sequence.sample(num_runs + 1, len_params)
# project the sampling results onto the parameter space
for i in range(len_params):
    blower, bupper = parameters.loc[i, 'min'], parameters.loc[i, 'max']
    sobol_se[:, i] = sobol_se[:, i] * (bupper-blower) + blower
nodes = np.round(sobol_se[1:], 4)

In [13]:
f_dir = '../output/Run0915/'
f_name = 'samples.csv'
df = pd.DataFrame(data=nodes, index=np.arange(nodes.shape[0]), columns=parameters.loc[:, 'Veneer_name'])
df.index.name = 'index'
df.to_csv(f'{f_dir}{f_name}', index=True)

In [28]:
# import samples
f_dir = '../output/Run0915/'
f_name = 'samples.csv'
nodes = pd.read_csv('{}{}'.format(f_dir, f_name), index_col='index')
nodes = nodes.values

#End

In [27]:
# create the parameter dataset with values adjusted
# assign distributions to parameters
# param_file = '../input/upper v1.csv'
# parameters = sr.load_parameter_file(param_file) 
# parameters = parameters[:-1]
# len_params = len(parameters)
# # process parameters and store initial values
# param_names, param_vename_dic, param_vename, param_types = sr.group_parameters(parameters)
# initial_params = get_initial_param_vals(vs[0], param_names, param_vename, param_vename_dic)
# # import samples
# fname = '../output/MW_results/samples.csv'
# samples = pd.read_csv(fname, index_col='id')
# samples = samples.T.values[:, -100:]
# # for i in range(index_adjust.shape[0]):
# #     ind_temp = index_adjust[i]
# #     samples[ind_temp, :] = np.product(samples[[ind_temp, *index_linear], :], axis=0)
# #     samples[index_linear[i], :] = [1]

# samples[19] = np.product(samples[[19, 20], :], axis=0)
# samples[6] = np.product(samples[[5, 6, 7], :], axis=0)
# samples[1, :] = np.product(samples[[0, 1, 2, 3, 9, 10, 11, 16, 17, 18], :], axis=0)
# # samples[1, :] = np.product(samples[[0, 1, 2], :], axis=0)
# cols_delete = [0, 2, 3, 9, 10, 11, 16, 17, 18, 5, 7, 20]
# samples[cols_delete, :] = [1]
        
# nodes = samples
# num_runs = nodes.shape[1]
# create samples
# nodes = np.ones((22, 50))
# nodes[1, :] = np.round(np.random.uniform(0.5, 1.5, size=nodes.shape[1]), 4)
# num_runs = nodes.shape[1]
# nodes = np.delete(nodes, cols_delete, axis=0)
# nodes.shape

#End Adapting the parameters

In [29]:
nodes = nodes[325:327]

In [30]:
num_runs = nodes.shape[0]
len_params = nodes.shape[1]

In [31]:
nodes.shape

(2, 22)

In [32]:
%%time
tss_results = []
total_runs = 0
group_loops = np.floor_divide(num_runs, num_copies) + 1
# tss_results = tss_results[:, 0:80]
for i in range(group_loops):
    group_run_responses = []
    if i == (group_loops - 1):
        num_copies_loop = (num_runs - i * num_copies)
    else:
        num_copies_loop = num_copies
    for j in range(num_copies_loop):
        total_runs += 1
        scaling_factor_index =  i * num_copies + j
#         print(scaling_factor_index)
        if scaling_factor_index >= num_runs:
            break
        scaling_factor = nodes[scaling_factor_index, :]
#         import ipdb
#         ipdb.set_trace()
        v= vs[j]
        v.drop_all_runs()
        
        # set scaling_factor to the current model
        for k in range(len_params):
            name = parameters.Veneer_name[k]
            param_new_factor = scaling_factor[k]
            param_value_ini = initial_params[name]
            if param_types[k] == 0:
                param_value_new = [param_new_factor for value in initial_params[name]]
            else:
                param_value_new = [param_new_factor * value for value in initial_params[name]]
            #set parameter values
            if name in param_vename_dic[param_vename[0]]:
                assert v.model.catchment.generation.set_param_values(name, param_value_new, fromList = True)
            if name in  param_vename_dic[param_vename[1]]:
                assert v.model.link.constituents.set_param_values(name, param_value_new,fromList = True)
            if name in  param_vename_dic[param_vename[2]]:
                assert v.model.node.set_param_values(name, param_value_new,fromList = True)
            if name in  param_vename_dic[param_vename[3]]:
                assert v.model.link.routing.set_param_values(name, param_value_new,fromList = True)

        response = v.run_model(start=start_date, end=end_date, run_async=True)
        group_run_responses.append(response)
        
    # Retrieve the time series of fine sediment mass at the interested gauge
    for j in range(num_copies_loop):
        scaling_factor_index =  i * num_copies + j
        if scaling_factor_index >= (num_runs):
            break
            
        v = vs[j]
        r = group_run_responses[j]   
        code = r.getresponse().getcode() # wait until the job finished   
        run_results = v.retrieve_multiple_time_series(criteria=ts_match_criteria, name_fn=veneer.name_for_variable)
        # store the daily results and the index of sampling
        result_np =  run_results.values
        try:
            tss_results = np.hstack((tss_results, result_np))
        except (ValueError, UnboundLocalError) as e:
            timeframe = np.array(run_results.index.date.tolist()).reshape(len(run_results.index), 1)
            tss_results = np.hstack((timeframe, result_np))
    if ((total_runs % 100) == 0 | (total_runs == num_runs)):
        num_write = tss_results.shape[1]
        np.savetxt('{}{}{}'.format(f_dir, str(total_runs), '_Tss_124001B.csv'), 
           tss_results, delimiter=',', newline='\n',
           header=", ".join(["Date"] + [str(i+1) for i in range(num_write)]), fmt='%s')
        tss_results = []
    veneer.log('Completed %d runs'%total_runs)

Completed 2 runs
Wall time: 2min 41s


In [26]:
total_runs

60

In [33]:
num_write = tss_results.shape[1]
np.savetxt('{}{}{}'.format(f_dir, str(total_runs), '_Tss_124001B.csv'), 
   tss_results, delimiter=',', newline='\n',
   header=", ".join(["Date"] + [str(i+1) for i in range(num_write)]), fmt='%s')

In [70]:
num_write = tss_results.shape[1]
# f_dir = '../output/linear_dep/'
np.savetxt('{}{}'.format(f_dir, 'temp_477.csv'), 
    tss_results, delimiter=',', newline='\n',
    header=", ".join(["Date"] + [str(i+1) for i in range(num_write)]), fmt='%s')

In [ ]:
%%time
for v in vs:    
    for k in range(len_params):
        name = parameters.Veneer_name[k]
        ini_param_val = initial_params[name]
        if name in param_vename_dic[param_vename[0]]: 
            v.model.catchment.generation.set_param_values(name, ini_param_val, fromList=True)
        if name in param_vename_dic[param_vename[1]]:
            v.model.link.constituents.set_param_values(name, ini_param_val, fromList=True)
        if name in param_vename_dic[param_vename[2]]:
            v.model.node.set_param_values(name, ini_param_val, fromList=True)
        if name in param_vename_dic[param_vename[3]]:
            v.model.link.routing.set_param_values(name, ini_param_val, fromList=True)
        if name in param_vename_dic[param_vename[4]]:
            v.model.node.constituents.set_param_values(name, ini_param_val, fromList=True, 
                                        node_types=['StorageNodeModel'],aspect='model')

In [ ]:
# Terminate the veneer servers
kill_all_now(processes)